In [1]:
import time

import pickle
import pandas as pd

from tqdm import tqdm
import numpy as np

file cleaned_data.sav from step 1.Pre.ipynb

In [8]:
in_file  = "./cleaned_data.sav"
df = pd.read_pickle(open(in_file, 'rb'))

In [10]:
df_split = np.array_split(df, 10)
for i in tqdm(range(len(df_split))):
    body_text = []
    for article in df_split[i]['body_text']:
        body_text.append(article)
    pickle.dump(body_text, open('./body_text/body_text_'+str(i)+'.sav', 'wb'))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [10:32<00:00, 63.22s/it]


sav to json

In [2]:
import spacy
import json

nlp = spacy.load('en_core_web_lg')
nlp.max_length = 2000000

def to_sent(text):
    doc=nlp(text)
    result=[]
    for sent in list(doc.sents):
        sent=str(sent)
        sent = sent.rstrip() #remove end space
        result.append(sent)
    return result

def to_json(df, save_file_name):
    results = []
    
    for text in tqdm(df):
        results += to_sent(text)
    
    json_file = save_file_name + '.json'
    with open(json_file,'w') as f:
        json.dump(results, f)

In [3]:
# abstract
abstract = df['abstract']

In [ ]:
to_json(abstract, 'abstract')

In [3]:
# body_text
filename = '0' # 1 2 3 4 ... 9
df = pd.read_pickle(open('./body_text/body_text_'+filename+'.sav', 'rb'))
to_json(df, './body_text/body_text_'+filename+'.json')

minie

In [118]:
# in file Minie_Bash.ipynb

filter and label sentence for biobert

In [13]:
pronounsList = ["another", "anybody", "anyone", "anything", "each", "either", "enough", "everybody", "everyone",
                "everything", "little", "much", "neither", "nobody", "no one", "nothing", "one", "other", "somebody",
                "something", "both", "few", "fewer", "many", "others", "several", "all", "any", "more", "most",
                "someone", "none", "some", "such", "I", "me", "my", "mine", "you", "your", "yours", "yourself",
                "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "myself", "it", "its", 
                "itself", "we", "us", "our", "ours", "ourselves", "they", "them", "their", "theirs", "themselves"]

def label(csv_file, name):
    csv_data = pd.read_csv(csv_file,encoding='utf-8')
    csv_df = pd.DataFrame(csv_data)
    
    # remove QUANT_
    df = csv_df.dropna()[~csv_df.dropna()['subject'].str.contains('QUANT_')]
    df = df.dropna()[~df.dropna()['object'].str.contains('QUANT_')]
    
    # remove single pronoun
    df = df.dropna()[~df.dropna()['subject'].str.lower().isin(pronounsList)]
    df = df.dropna()[~df.dropna()['object'].str.lower().isin(pronounsList)]
    
    # remove \n
    df['sentence'] = df['sentence'].replace(r'\s+|\\n', ' ', regex=True)
    
    # label sent
    def label1(sub,obj,sent):
        sub_location = sent.find(sub)
        if(sub_location == -1):
            return sent
        new_sub = '@' + sub.upper() + '#'
        sent = sent[:sub_location] + new_sub + sent[sub_location+len(sub):]
        obj_location = sent.find(obj)
        if(obj_location == -1):
            return sent
        new_obj = '@' + obj.upper() + '#'
        sent = sent[:obj_location] + new_obj + sent[obj_location+len(obj):]
        return sent
    df['sent'] = [ label1(a,b,c) for a,b,c in zip(df['subject'],df['object'],df['sentence']) ]
    
    # set label
    def label2(a):
        if(a[0].count('@') >= 2 and a[0].count('#') >= 2):
            return 1
        else:
            return 0
    df['label'] = [ label2(a) for a in zip(df['sent']) ]
    
    # save all
    df.insert(0, 'index', range(len(df)), allow_duplicates=False)
    df.to_csv('./for_biobert/'+ 'labeled_'+csv_file , index=False)
    
    # save for test.tsv
    df=df.drop('subject',axis=1)
    df=df.drop('relation',axis=1)
    df=df.drop('object',axis=1)
    df=df.drop('sentence',axis=1)
    df.columns = ['index','sentence','label']
    
    df.to_csv('./for_biobert/'+ name + '_test.tsv', sep="\t" , index=False)
    

In [18]:
label('body_text_10.json.csv', 'body_text_10')

run biobert

In [119]:
# in file ./biobert/Biobert_Bash.ipynb

In [7]:
# read tsv
tsv = pd.read_csv('./for_biobert/body_text_1_test.tsv', sep='\t')

In [3]:
# read result_csv
result_csv = pd.read_csv('./for_biobert/body_text_1_test_result差.csv',encoding='utf-8')
result_csv3 = pd.read_csv('./for_biobert/body_text_3_test_result差.csv',encoding='utf-8')
result_csv4 = pd.read_csv('./for_biobert/body_text_4_test_result差.csv',encoding='utf-8')

In [6]:
# read labeled_csv
labeled_csv = pd.read_csv('./for_biobert/labeled_body_text_1.json.csv',encoding='utf-8')

,probabilities,index,label
0,[0.44310427 0.5568958 ],0,True
1,[0.41895014 0.5810498 ],1,True
2,[0.46260262 0.5373974 ],2,True
3,[0.57194597 0.42805406],3,False
4,[0.57194597 0.42805406],4,False
...,...,...,...
7720092,[0.37074366 0.6292563 ],7720092,True
7720093,[0.35885397 0.64114606],7720093,True
7720094,[0.35885397 0.64114606],7720094,True
7720095,[0.35885397 0.64114606],7720095,True
